# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
# from unityagents import UnityEnvironment
# import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
# env = UnityEnvironment(file_name="Banana_Linux/Banana.x86_64")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# # get the default brain
# brain_name = env.brain_names[0]
# brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# # reset the environment
# env_info = env.reset(train_mode=True)[brain_name]

# # number of agents in the environment
# print('Number of agents:', len(env_info.agents))

# # number of actions
# action_size = brain.vector_action_space_size
# print('Number of actions:', action_size)

# # examine the state space 
# state = env_info.vector_observations[0]
# print(type(state))
# print('States look like:', state)
# state_size = len(state)
# print('States have length:', state_size)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
# env_info = env.reset(train_mode=False)[brain_name] # reset the environment
# state = env_info.vector_observations[0]            # get the current state
# score = 0                                          # initialize the score
# while True:
#     action = np.random.randint(action_size)        # select an action
#     env_info = env.step(action)[brain_name]        # send the action to the environment
#     next_state = env_info.vector_observations[0]   # get the next state
#     reward = env_info.rewards[0]                   # get the reward
#     done = env_info.local_done[0]                  # see if episode has finished
#     score += reward                                # update the score
#     state = next_state                             # roll over the state to next time step
#     if done:                                       # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))

When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
import numpy as np
import os
import pickle
import torch
from collections import deque
from typing import List

from unityagents import UnityEnvironment

from agent import Agent
from scaler import Scaler

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
distance_scaler = Scaler(
    min_value=0.0,
    max_value=1.0,
    num_buckets=10,
)

velocity_scaler = Scaler(
    min_value=-20.0,
    max_value=+20.0,
    num_buckets=10,
)

state_size = 7 * (4 + distance_scaler.num_buckets) + 2 * velocity_scaler.num_buckets

def encode_state(state: np.ndarray) -> np.ndarray:
    state = state.tolist()  # convert np.ndarray to List[int]
    encoded_state = []
    for idx in range(0, 35, 5):
        encoded_state += state[idx: idx + 4]
        distance_features = [0 for _ in range(distance_scaler.num_buckets)]
        distance_features[distance_scaler.scale(state[idx + 4])] = 1
        encoded_state += distance_features
        
    for idx in range(35, 37):
        velocity_features = [0 for _ in range(velocity_scaler.num_buckets)]
        velocity_features[velocity_scaler.scale(state[idx])] = 1
        encoded_state += velocity_features
        
    return np.array(encoded_state)

In [10]:
def train(
    env,
    brain_name: str,
    agent: Agent,
    n_episodes: int=2000,
    max_timestamp_per_episode: int=1000,
    eps_start: float=1.0,
    eps_end: float=0.01,
    eps_decay: float=0.995,
):
    scores = []
    scores_window = deque(maxlen=100)
    eps = eps_start
    
    for episode_idx in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        encoded_state = encode_state(state)
        score = 0
        for timestamp_idx in range(max_timestamp_per_episode):
            action = agent.act(encoded_state, eps)
            env_info = env.step(action)[brain_name]
            (
                next_state,
                reward,
                done,
            ) = (
                env_info.vector_observations[0],
                env_info.rewards[0],
                env_info.local_done[0],
            )
            encoded_next_state = encode_state(next_state)
            agent.step(encoded_state, action, reward, encoded_next_state, done)
            encoded_state = encoded_next_state
            score += reward
            if done:
                break
        
        scores_window.append(score)
        scores.append(score)
        eps = max(eps_end, eps * eps_decay)
        average_score = np.mean(scores_window)
        
        if episode_idx % 100 == 0:
            print(f"Episode {episode_idx} -- Average score: {average_score}")
            
        if average_score >= 20.0:
            print(f"Problem solved within {episode_idx} episodes.")
            print(f"Average score: {average_score}")
            break
    return 

In [11]:
def inference(
    env,
    brain_name: str,
    agent: Agent,
    eps: float=0.01,  # use the eps_end in training
) -> None:
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations[0]
    encoded_state = encode_state(state)
    score = 0
    while True:
        action = agent.act(encoded_state, eps)
        env_info = env.step(action)[brain_name]
        (
            next_state,
            reward,
            done,
        ) = (
            env_info.vector_observations[0],
            env_info.rewards[0],
            env_info.local_done[0],
        )
        encoded_next_state = encode_state(next_state)
        encoded_state = encoded_next_state
        score += reward
        if done:
            break

    print("Score: {}".format(score))

In [12]:
env = UnityEnvironment(file_name="Banana_Linux/Banana.x86_64")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [13]:
OVERWRITE = True
QNETWORK_LOCAL_PICKLE_FILE_PATH = "qnetwork_local.pt"

agent = Agent(
    state_size=state_size,
    action_size=brain.vector_action_space_size,
    seed=514,
)

if not OVERWRITE and os.path.exists(QNETWORK_LOCAL_PICKLE_FILE_PATH):
    agent.qnetwork_local.load_state_dict(torch.load(QNETWORK_LOCAL_PICKLE_FILE_PATH))
else:
    agent = Agent(
        state_size=state_size,
        action_size=brain.vector_action_space_size,
        seed=514,
    )
    train(env, brain_name, agent)
    torch.save(agent.qnetwork_local.state_dict(), QNETWORK_LOCAL_PICKLE_FILE_PATH)

Episode 100 -- Average score: 0.22
Episode 200 -- Average score: 2.13
Episode 300 -- Average score: 4.82
Episode 400 -- Average score: 7.69
Episode 500 -- Average score: 9.91
Episode 600 -- Average score: 11.66
Episode 700 -- Average score: 13.54
Episode 800 -- Average score: 14.13
Episode 900 -- Average score: 14.77
Episode 1000 -- Average score: 14.78
Episode 1100 -- Average score: 13.88
Episode 1200 -- Average score: 14.13
Episode 1300 -- Average score: 15.74
Episode 1400 -- Average score: 15.81
Episode 1500 -- Average score: 16.15
Episode 1600 -- Average score: 15.31
Episode 1700 -- Average score: 16.12
Episode 1800 -- Average score: 15.11
Episode 1900 -- Average score: 15.14
Episode 2000 -- Average score: 14.24


In [14]:
inference(env, brain_name, agent)

Score: 15.0


In [15]:
env.close()